In [8]:
# -*- coding: utf-8 -*-
import os
import sys

from sqlalchemy import Column, ForeignKey, Integer, CHAR, Date, String, Time, Index, DateTime, TIMESTAMP, func
from sqlalchemy.dialects.mysql import INTEGER, BIT, TINYINT, TIME, DOUBLE, TEXT
from sqlalchemy.ext.declarative import declarative_base

import datetime
from sqlalchemy import create_engine
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func, Float


server = 'ec2-35-161-71-119.us-west-2.compute.amazonaws.com'
connection_string = 'mysql+mysqldb://root:gozjqkqh1@{}:3306/Test'.format(server)
engine = create_engine(connection_string, pool_recycle = 3600, encoding='utf-8')
Session = sessionmaker(bind=engine)
Base = declarative_base()

class News(Base):
    __tablename__ = 'naver'

    link            = Column(CHAR(100), primary_key = True, nullable = False)
    title           = Column(TEXT(100), nullable = False)
    content         = Column(TEXT(1000), nullable = False)
    crawl_time      = Column(DateTime, nullable = False)

class Comment(Base):
    __tablename__ = 'comment'

    url           = Column(TEXT(1000), nullable = False)
    comment       = Column(TEXT(1000), nullable = False)
    __table_args__       = (PrimaryKeyConstraint('url', 'comment'), {},)

class Similarity(Base):
    __tablename__ = 'similar'

    first_link            = Column(CHAR(100), nullable = False)
    second_link           = Column(CHAR(100), nullable = False)
    Similarity            = Column(Float, nullable = False)
    __table_args__       = (PrimaryKeyConstraint('first_link', 'second_link'), {},)

In [10]:
session = Session()
result = session.query(News.link, News.content).all()
data = []
for row in result:
    news = {}
    news['url'] = row.link
    news['content'] = row.content.decode('utf-8')
    data.append(news)
    
from konlpy.utils import pprint
from konlpy.tag import Kkma
kkma = Kkma()
noun_data = []
for da in data:
    nouns = {}
    nouns['nouns'] = kkma.nouns(da['content'])
    nouns['url'] = da['url']
    noun_data.append(nouns)
    
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(ve1, ve2):
    vec1 = Counter(ve1)
    vec2 = Counter(ve2)
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    

import itertools
final_result = []
for i in itertools.combinations(noun_data, r = 2):
    data = {}
    ve1 = i[0]['nouns']
    ve2 = i[1]['nouns']
    cosine_d = get_cosine(ve1, ve2)
    
    data['similarity'] = cosine_d
    data['first_link'] = i[0]['url']
    data['second_link'] = i[1]['url']
    final_result.append(data)
    

session = Session()
for i in final_result:
    try:
        cos_sim = Similarity(first_link = i['first_link'], second_link = i['second_link'], Similarity = i['similarity'])
        session.add(cos_sim)
    except:
        continue
session.commit()
session.close()